**Imports:**

In [1]:
from types import SimpleNamespace
from copy import deepcopy

import numpy as np
from scipy import optimize

import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

import ipywidgets as widgets

DO_PRINT = False

# Utility function

In [2]:
def utility_func(x1,x2,alpha,beta):

    with np.errstate(divide='ignore', invalid='ignore'):

        if beta == 0:
            return x1**alpha*x2**(1-alpha)
        else:
            return (alpha*x1**(-beta)+(1-alpha)*x2**(-beta))**(-1/beta)

def indiff_func_x1(u0,x1,alpha,beta):
    if beta == 0.0:
        return (u0/x1**alpha)**(1/(1-alpha))
    else:
        return ((u0**(-beta)-alpha*x1**(-beta))/(1-alpha))**(-1/beta)
    
def indiff_func_x2(u0,x2,alpha,beta):
    if beta == 0.0:
        return (u0/x2*(1-alpha))**(1/alpha)
    else:
        return ((u0**(-beta)-(1-alpha)*x2**(-beta))/alpha)**(-1/beta)
    

# Maximize utility

In [3]:
def maximize_utility(p1,p2,m,alpha,beta):
    
    # a. target
    def target_2d(x):

        excess_spending = p1*x[0]+p2*x[1]-m
        return -utility_func(x[0],x[1],alpha,beta) + 1000*np.max([excess_spending,-x[0],-x[1],0])

    # b. solve
    x_guess = np.array([m/p1,m/p2])/2
    res = optimize.minimize(target_2d,x_guess,method="Nelder-Mead")

    return res.x[0],res.x[1],-res.fun


# Minimize cost

In [4]:
def minimize_cost(x_guess,u0,p1,p2,alpha,beta):
    
    # a. target
    def target_2d(x):
        x1 = x[0]
        x2 = x[1]
        udiff = (utility_func(x1,x2,alpha,beta)-u0)** 2

        return (p1*x1 + p2*x2 + 1000*udiff + 1000*np.max([-x[0],-x[1],0]))

    # solve
    res = optimize.minimize(target_2d,x_guess,method="Nelder-Mead")

    return res.x[0],res.x[1],res.fun

# Plot indifference curve

In [5]:
def plot_indifference_curve(ax,u0,alpha,beta,x1_max=10,x2_max=10,N=100,ls='-',color='black'):
    
    # a. fix x1
    x1_x1 = np.linspace(0,x1_max,N)
    with np.errstate(divide='ignore', invalid='ignore'):
        x2_x1 = indiff_func_x1(u0,x1_x1,alpha,beta)

    # b. fix x2
    x2_x2 = np.linspace(0,x2_max,N)
    with np.errstate(divide='ignore', invalid='ignore'):
        x1_x2 = indiff_func_x2(u0,x2_x2, alpha, beta)

    # c. combine
    x1 = np.hstack([x1_x1,x1_x2])
    x2 = np.hstack([x2_x1,x2_x2])

    # . clean
    with np.errstate(divide='ignore', invalid='ignore'):
        u0s = utility_func(x1,x2,alpha,beta)
        
    I = np.isclose(u0s, u0)
    x1 = x1[I]
    x2 = x2[I]

    I = (x1 >= 0) & (x2 >= 0)
    x1 = x1[I]
    x2 = x2[I]

    # e. sort
    I = np.argsort(x1)
    x1 = x1[I]
    x2 = x2[I]

    ax.plot(x1,x2,ls=ls,color=color)
    

# Plot decomposition

In [6]:
def plot_decomposition_exo(
    method='Hicks',p1=1.0,p1_new=2.0,p2=1.0,m=20.0,alpha=0.5,beta=0.0,
    steps=3,x1_max=25,x2_max=25,N=100,name=None
):
    
    fig = plt.figure(figsize=(6,6),dpi=100)
    ax = fig.add_subplot(1,1,1)
    
    # a. calculations
    X = maximize_utility(p1,p2,m,alpha,beta)
    x1,x2,u_ast = X

    Z = maximize_utility(p1_new,p2,m,alpha,beta)
    x1_new,x2_new,u_ast_new = Z
    
    if method == 'Hicks':
        Y = minimize_cost((x1,x2),u_ast,p1_new,p2,alpha,beta)
        x1_alt,x2_alt,m_alt = Y
    else:
        m_alt = p1_new*x1 + p2*x2
        Y = maximize_utility(p1_new,p2,m_alt,alpha,beta)
        x1_alt,x2_alt,u_ast_alt = Y
        
    
    # b. plots
    ax.plot([0,m/p1],[m/p2,0],ls='-',label='original',color=colors[0])
    if steps > 1:
        ax.plot([0,m/p1_new],[m/p2,0],ls='-',label='final',color=colors[1])
    if steps > 2:
        ax.plot([0,m_alt/p1_new],[m_alt/p2,0],ls='-',label='compensated',color=colors[2])
        
    # X
    ax.plot(x1,x2,'ro',color='black')
    ax.text(x1*1.015,x2*1.015,'$X$')
    
    plot_indifference_curve(
        ax,u_ast,alpha,beta,x1_max,x2_max,N,ls='--',color=colors[0]
    )

    # Y
    if steps > 2:
        
        ax.plot(x1_alt,x2_alt,'ro',color='black')
        ax.text(x1_alt*1.015,x2_alt*1.015,'$Y$')
        
        if not method == 'Hicks':
            plot_indifference_curve(
                ax,u_ast_alt,alpha,beta,x1_max,x2_max,N,ls='--',color=colors[2]
            )
        
    # Z
    if steps > 1:
        
        ax.plot(x1_new,x2_new,'ro',color='black')
        ax.text(x1_new*1.02,x2_new*1.02,'$Z$')
        plot_indifference_curve(
            ax,u_ast_new,alpha,beta,x1_max,x2_max,N,ls='--',color=colors[1]
        )

    if steps > 2:
        
        line = f'subtitution: $Y-X$ = ({x1_alt-x1:5.2f},{x2_alt-x2:5.2f})\n'
        line += f'income: $Z-Y$ = ({x1_new-x1_alt:5.2f},{x2_new-x2_alt:5.2f})\n'
        ax.text(0.55*x1_max,0.87*x2_max,line,backgroundcolor='white')
    
    # c. details
    ax.set_xlim([0,x1_max])
    ax.set_ylim([0,x2_max])

    ax.legend(frameon=True,loc='right')

    fig.tight_layout()
    if not name is None:
        fig.savefig(name)

In [7]:
def plot_decomposition_endo(
    method='Hicks',p1=1.0,p1_new=2.0,p2=1.0,w1=12.0,w2=8.0,alpha=0.5,beta=0.0,
    steps=4,x1_max=25,x2_max=25,N=100,name=None):

    fig = plt.figure(figsize=(6,6),dpi=100)
    ax = fig.add_subplot(1,1,1)
    
    m = p1*w1+p2*w2
    m_new = p1_new*w1+p2*w2

    # a. calculations
    X = maximize_utility(p1,p2,m,alpha,beta)
    x1,x2,u_ast = X

    Z1 = maximize_utility(p1_new,p2,m,alpha,beta)
    x1_fixm,x2_fixm,u_ast_fixm = Z1

    #Y = minimize_cost((x1,x2),u_ast,p1_new,p2,alpha,beta)
    #x1_alt,x2_alt,m_alt = Y
    
    if method == 'Hicks':
        Y = minimize_cost((x1,x2),u_ast,p1_new,p2,alpha,beta)
        x1_alt,x2_alt,m_alt = Y
    else:
        m_alt = p1_new*x1 + p2*x2
        Y = maximize_utility(p1_new,p2,m_alt,alpha,beta)
        x1_alt,x2_alt,u_ast_alt = Y
        
    Z2 = maximize_utility(p1_new,p2,m_new,alpha,beta)
    x1_new,x2_new,u_ast_new = Z2

    # b. plots
    ax.plot(w1,w2,'ro',label='endowment',color='black')    
    ax.plot([0,m/p1],[m/p2,0],ls='-',label='original',color=colors[0])
    if steps > 1:
        ax.plot([0,m_new/p1_new],[m_new/p2,0],ls='-',label='final',color=colors[1])
    if steps > 2:
        ax.plot([0,m_alt/p1_new],[m_alt/p2,0],ls='-',label='compensated',color=colors[2])
    if steps > 3:
        ax.plot([0,m/p1_new],[m/p2,0],ls='-',label='constant income',color=colors[2])

    # X
    ax.plot(x1,x2,'ro',color='black')
    ax.text(x1*1.015,x2*1.015,'$X$')
    plot_indifference_curve(
        ax,u_ast,alpha,beta,x1_max,x2_max,N,ls='--',color=colors[0]
    )

    # Y
    if steps > 2:
        
        ax.plot(x1_alt,x2_alt,'ro',color='black')
        ax.text(x1_alt*1.015,x2_alt*1.015,'$Y$')
        
        if not method == 'Hicks':
            plot_indifference_curve(
                ax,u_ast_alt,alpha,beta,x1_max,x2_max,N,ls='--',color=colors[2]
            )        

    # Z2
    if steps > 1:
        ax.plot(x1_new,x2_new,'ro',color='black')
        ax.text(x1_new*1.02,x2_new*1.02,'$Z_2$')
        plot_indifference_curve(
            ax,u_ast_new,alpha,beta,x1_max,x2_max,N,ls='--',color=colors[1]
        )

    # Z1
    if steps > 3:
        ax.plot(x1_fixm,x2_fixm,'ro',color='black')
        ax.text(x1_fixm*1.02,x2_fixm*1.02,f'$Z_1$')
        plot_indifference_curve(
            ax,u_ast_fixm,alpha,beta,x1_max,x2_max,N,ls='--',color=colors[3]
        )
    
    if steps > 3:
        line = f'subtitution: $Y-X$ = ({x1_alt-x1:5.2f},{x2_alt-x2:5.2f})\n'
        line += f'income: $Z_1-Y$ = ({x1_fixm-x1_alt:5.2f},{x2_fixm-x2_alt:5.2f})\n'
        line += f'wealth: $Z_2-Z_1$ = ({x1_new-x1_fixm:5.2f},{x2_new-x2_fixm:5.2f})'
        ax.text(0.55*x1_max,0.87*x2_max,line,backgroundcolor='white')

    # c. details
    ax.set_xlim([0,x1_max])
    ax.set_ylim([0,x2_max])
    
    ax.legend(frameon=True,loc='right')
    fig.tight_layout()
    
    if not name is None:
        fig.savefig(name)


# Static

In [8]:
for steps in [1,2,3]:
    if DO_PRINT: plot_decomposition_exo(method='Slutsky',steps=steps,name=f'CobbDouglas_Slutsky_{steps}.pdf');

In [9]:
for steps in [1,2,3]:
    if DO_PRINT: plot_decomposition_exo(method='Hicks',steps=steps,name=f'CobbDouglas_Hicks_{steps}.pdf');

In [10]:
for steps in [1,2,3,4]:
    if DO_PRINT: plot_decomposition_endo(steps=steps,name=f'CobbDouglas_Hicks_endo_{steps}.pdf');

# Interactive - exogenous income

$$
u(x_1,x_2) = (\alpha x_1^{-\beta}+(1-\alpha)x_2^{-\beta})^{-1/\beta}
$$

In [11]:
widgets.interact(lambda method,alpha,beta,p1_new,steps: 
                 plot_decomposition_exo(method=method,alpha=alpha,beta=beta,p1_new=p1_new,steps=steps),
    method=widgets.RadioButtons(options=['Slutsky','Hicks'],description='method:',disabled=False),
    alpha=widgets.FloatSlider(description=r'alpha',min=0.1, max=1.0, step=0.05, value=0.50), 
    beta=widgets.FloatSlider(description=r'beta',min=-0.95, max=5.0, step=0.05, value=0.0),                    
    p1_new=widgets.FloatSlider(description=r'p1_new',min=0.1,max=5.0,step=0.05,value=2.0),
    steps=widgets.IntSlider(description=r'step',min=1,max=3,value=3),
);

interactive(children=(RadioButtons(description='method:', options=('Slutsky', 'Hicks'), value='Slutsky'), Floa…

# Interactive - endogenous income

$$
u(x_1,x_2) = (\alpha x_1^{-\beta}+(1-\alpha)x_2^{-\beta})^{-1/\beta}
$$

In [12]:
widgets.interact(lambda method,alpha,beta,p1_new,steps: 
                 plot_decomposition_endo(method=method,alpha=alpha,beta=beta,p1_new=p1_new,steps=steps),
    method=widgets.RadioButtons(options=['Slutsky','Hicks'],description='method:',disabled=False),
    alpha=widgets.FloatSlider(description=r'alpha',min=0.1, max=1.0, step=0.05, value=0.50), 
    beta=widgets.FloatSlider(description=r'beta',min=-0.95, max=5.0, step=0.05, value=0.0),                    
    p1_new=widgets.FloatSlider(description=r'p1_new',min=0.1,max=5.0,step=0.05,value=2.0), 
    steps=widgets.IntSlider(description=r'step',min=1,max=4,value=4),                 
);

interactive(children=(RadioButtons(description='method:', options=('Slutsky', 'Hicks'), value='Slutsky'), Floa…